In [30]:
import pandas as pd
import tweepy
from tweepy import OAuthHandler
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
import re
import datetime

In [27]:
%run ./keys.ipynb
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [114]:
df = pd.read_csv('../../../../sarahr/Ironhack/political_spanish_sentiment/political_spanish_sentiment_scripts/political_spanish_sentiment_ddbb.csv')

In [115]:
df

,Unnamed: 0,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,partido,tipo de post,hashtags,campaña
0,0,Sarabia De Boj,2021-04-28 13:35:34+00:00,1387400098469064706,@FreedomWarro @Valentin_GarAl @asancfisac @Idi...,"[173, 215]",['Twitter for Android'],0,0,7,259,16,Partido Popular,mención,[],general
1,1,jesu lopez lo,2021-04-28 13:35:20+00:00,1387400036624080899,Toni Cantó ese filósofo que estableció el teo...,"[0, 215]",['Twitter for iPhone'],0,0,273,272,4480,Partido Popular,mención,[],general
2,2,Miguel Benito,2021-04-28 13:35:14+00:00,1387400014406852611,"@populares @IdiazAyuso ¿""El mejor hospital de ...","[23, 281]",['Twitter Web App'],0,0,145,225,23866,Partido Popular,mención,[],general
3,3,Bernardo Diaz Martínez,2021-04-28 13:35:08+00:00,1387399988846776322,@publico_es No están ellos sacando todos los d...,"[12, 192]",['Twitter Web App'],0,0,0,87,310,Partido Popular,mención,[],general
4,4,Ritxi ·🔻·,2021-04-28 13:35:03+00:00,1387399968097509377,"@AngelBlancoMo @populares Bueno, yo hablaba de...","[26, 304]",['Twitter for Android'],0,0,63,346,1130,Partido Popular,mención,[],general
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58536,9995,Red Por España,2021-04-26 19:26:13+00:00,1386763565173022727,😡 La vergonzosa equidistancia que Edmundo Bal ...,"[0, 134]",TweetDeck,1,1,11911,12974,13703,VOX,mención,[],general
58537,9996,Martita,2021-04-26 19:26:13+00:00,1386763563595976705,@Nela44636322 @m_ntesa_vlc__tw Dejate de gilip...,"[31, 101]",Twitter for Android,0,1,163,301,2741,VOX,mención,[],general
58538,9997,Murciélago,2021-04-26 19:26:11+00:00,1386763556880855046,@fsierra VOX alienta y hasta las personas con ...,"[9, 70]",Twitter for iPhone,0,0,1008,2860,81531,VOX,mención,[],general
58539,9998,Ivanchu 🇪🇸🔸🇪🇺,2021-04-26 19:26:11+00:00,1386763556411125763,@CiudadanoEspa14 @IdiazAyuso @PabloIglesias @C...,"[112, 205]",Twitter for iPhone,3,18,733,1212,19023,VOX,mención,[],general


In [106]:
def extracting_tweets(df, account):
    """
    This function extract a data frame with the tweets of an account
    and filter the data frame with the columns selected.
    """
    
    # parameters
    last_tweet_id = max(df['id'])
    count = 10
    
    # extracting the tweets
    tweets = [tweet for tweet in tweepy.Cursor(api.user_timeline,
                                           screen_name=account,
                                           tweet_mode='extended',
                                           exclude_replies=True,
                                               max_id=last_tweet_id,
                                              include_rts=False).items(count)]
    # creating the data frame
    tweets_json = [tweet._json for tweet in tweets]
    df_tweets = pd.json_normalize(tweets_json)
    
    # selecting useful columns
    columns_selected = ['user.name', 'created_at', 'id', 'full_text', 'display_text_range', 
                    'source', 'retweet_count', 'favorite_count', 'user.followers_count', 
                    'user.friends_count', 'user.statuses_count']
    
    df_tweets_filtered = df_tweets[columns_selected]
    
    # cleaning date time
    df_tweets_filtered.loc[:, 'created_at'] = pd.to_datetime(df_tweets_filtered['created_at'])
    
    # cleaning source of the tweet
    list_sources = list(df_tweets_filtered['source'])
    df_tweets_filtered.loc[:, 'source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]
    
    return df_tweets_filtered

In [45]:
def extracting_mentions(query):
    """
    This function extract a data frame with the result of a query.
    """
    
    # date and amount information
    count = 10

    today = datetime.date.today()
    yest = today - datetime.timedelta(days=1)
    yest2 = today - datetime.timedelta(days=7)
    tomorrow = today + datetime.timedelta(days=1)
    since=yest2
    until=today
    
    print(f'Until: {until}')
    print(f'Since: {since}')
    
    # getting the mentions
    
    print(f'Getting mentions for {query} since {since} until {until}.')
    
    mentions = [tweet for tweet in tweepy.Cursor(api.search, 
                                                q=query + ' -filter:retweets', 
                                                lang="es", 
                                                tweet_mode='extended',
                                                 until=until,
                                                 since=since,
                                                result_type="recent").items(count)]
    mentions_json = [tweet._json for tweet in mentions]
    df_mentions = pd.json_normalize(mentions_json)
    
    # selecting useful columns
    columns_selected = ['user.name', 'created_at', 'id', 'full_text', 'display_text_range', 
                    'source', 'retweet_count', 'favorite_count', 'user.followers_count', 
                    'user.friends_count', 'user.statuses_count']
    
    df_mentions_filtered = df_mentions[columns_selected]
    
    # cleaning date time
    df_mentions_filtered.loc[:, 'created_at'] = pd.to_datetime(df_mentions_filtered['created_at'])
    
    # cleaning source of the tweet
    list_sources = list(df_mentions_filtered['source'])
    df_mentions_filtered.loc[:, 'source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]
    
    return df_mentions_filtered

In [107]:
df_pp_ayuso = extracting_mentions('ayuso')

Until: 2021-04-28
Since: 2021-04-21
Getting mentions for ayuso since 2021-04-21 until 2021-04-28.


/Users/sarahr/miniconda3/envs/ironhack_env/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/Users/sarahr/miniconda3/envs/ironhack_env/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [108]:
df_pp_ayuso

,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count
0,Yo 🇪🇸,2021-04-27 23:59:02+00:00,1387194608128516096,@PabloEchenique @PabloIglesias Esperando que c...,"[31, 97]",Twitter for iPhone,0,0,526,296,16204
1,Ignacio Caralt,2021-04-27 23:58:57+00:00,1387194590499852292,@JaviRodrigo11 La Opción 1: Ayuso y Monasterio...,"[15, 161]",Twitter Web App,0,3,50,92,3884
2,Desakordes,2021-04-27 23:58:45+00:00,1387194540172455937,@GaByNnia @Africamar @TercioRepublico A los TA...,"[38, 314]",Twitter for Android,2,4,2868,3213,17768
3,Jorge Villet Salas,2021-04-27 23:58:44+00:00,1387194533876805635,#27Abr #Internacionales\nAyuso supera a las tr...,"[0, 143]",Twitter Web App,0,0,12913,1216,67891
4,nosé,2021-04-27 23:58:39+00:00,1387194513970671616,Las risas que me voy a echar si gana ayuso des...,"[0, 116]",Twitter for Android,0,1,98,200,43018
5,Enrique rm529x7,2021-04-27 23:58:25+00:00,1387194454021492740,@Luis44544014 @Teofilo1952 Y en Madrid donde l...,"[27, 108]",Twitter Web App,0,1,97,140,4042
6,Antonio Ruiz Vaz,2021-04-27 23:58:24+00:00,1387194448547913730,Después de oír Ayuso y Monasterio todas las ba...,"[0, 280]",Twitter for Android,0,0,112,180,2895
7,LucasRCDE 🇪🇸🎲,2021-04-27 23:58:18+00:00,1387194424795467778,"Tengo la teoría de que, evidentemente, lo de A...","[0, 205]",Twitter for Android,0,0,436,196,16035
8,Santi Orozco 🧡🇪🇸 🇪🇺,2021-04-27 23:58:11+00:00,1387194396702121988,@mariviti1 Cs hizo la moción de censura en Mur...,"[11, 288]",Twitter for Android,1,1,2039,2702,31750
9,Ramón Troya,2021-04-27 23:58:05+00:00,1387194369376141318,"@NagoNura Por supuesto, no sólo es una amenaza...","[10, 149]",Twitter Web App,0,0,206,261,4124


In [99]:
df_pp = extracting_tweets(df_pp, 'populares')

<ipython-input-19-b0e29fe1aee5>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_filtered['created_at'] = pd.to_datetime(df_tweets_filtered['created_at'])
<ipython-input-19-b0e29fe1aee5>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_filtered['source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]


In [100]:
df_pp

,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count
0,Partido Popular,2021-04-27 21:42:12+00:00,1387160176663441411,"""No tengo nada que ver con el PSOE, no voy a p...","[0, 225]",[Twitter Media Studio - LiveCut],130,364,822199,5291,102814
1,Partido Popular,2021-04-27 21:37:40+00:00,1387159035825299458,“Les conviene esta tensión porque así no habla...,"[0, 278]",[Twitter Media Studio - LiveCut],120,276,822199,5291,102814
2,Partido Popular,2021-04-27 21:32:38+00:00,1387157767602024449,“Sánchez e Iglesias tenían un plan perfectamen...,"[0, 240]",[Twitter Media Studio - LiveCut],175,417,822199,5291,102814
3,Partido Popular,2021-04-27 20:59:41+00:00,1387149476192718850,"“Todo lo que ocurre en Madrid, trasciende a Ma...","[0, 225]",[Twitter Media Studio - LiveCut],60,101,822199,5291,102814
4,Partido Popular,2021-04-27 20:57:59+00:00,1387149048205955082,"""Me he dedicado a Madrid y a España, y a parti...","[0, 224]",[Twitter Media Studio - LiveCut],92,188,822199,5291,102814
...,...,...,...,...,...,...,...,...,...,...,...
95,Partido Popular,2021-04-24 22:20:54+00:00,1386082752294961152,"""No hay ninguna transparencia por parte del Go...","[0, 228]",[Twitter Web App],114,212,822199,5291,102814
96,Partido Popular,2021-04-24 22:07:57+00:00,1386079491722825728,"""Pablo Casado ha sido la persona que ha confia...","[0, 200]",[Twitter Web App],51,145,822199,5291,102814
97,Partido Popular,2021-04-24 21:49:49+00:00,1386074929251196928,"""Nuestro mayor problema ahora es que no llegan...","[0, 206]",[Twitter Web App],96,256,822199,5291,102814
98,Partido Popular,2021-04-24 21:44:34+00:00,1386073607068082181,"""Quiero gobernar en solitario"".\n\n""Me debo a ...","[0, 179]",[Twitter Web App],193,551,822199,5291,102814


In [105]:
df[df['id'] == 1387194608128516096]

,Unnamed: 0,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,partido,tipo de post,hashtags,campaña


In [74]:
list_names = list(df['user.name'])

In [75]:


df.loc[:,'user.name'] = [re.findall(r'(?i)\b[a-z]+\b', s) for s in list_names]

/Users/sarahr/miniconda3/envs/ironhack_env/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [86]:
df.loc[:,'user.name'] = df['user.name'].apply(lambda x: re.findall(r'(?i)\b[a-z]+\b', x))

In [88]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [91]:
df.loc[:,'user.name'] = df['user.name'].apply(remove_emoji)

In [97]:
df[df['user.name'] == 'CiudadanosCs']

,Unnamed: 0,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,partido,tipo de post,hashtags,campaña


In [109]:
df_pp_ayuso.to_csv('../political_spanish_sentiment_scripts/political_spanish_sentiment_ddbb.csv', mode='a', header=False)

In [111]:
pd.read_csv('../../../../sarahr/Ironhack/political_spanish_sentiment/political_spanish_sentiment_scripts/political_spanish_sentiment_ddbb.csv')

,Unnamed: 0,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,partido,tipo de post,hashtags,campaña
0,0,Sarabia De Boj,2021-04-28 13:35:34+00:00,1387400098469064706,@FreedomWarro @Valentin_GarAl @asancfisac @Idi...,"[173, 215]",['Twitter for Android'],0,0,7,259,16,Partido Popular,mención,[],general
1,1,jesu lopez lo,2021-04-28 13:35:20+00:00,1387400036624080899,Toni Cantó ese filósofo que estableció el teo...,"[0, 215]",['Twitter for iPhone'],0,0,273,272,4480,Partido Popular,mención,[],general
2,2,Miguel Benito,2021-04-28 13:35:14+00:00,1387400014406852611,"@populares @IdiazAyuso ¿""El mejor hospital de ...","[23, 281]",['Twitter Web App'],0,0,145,225,23866,Partido Popular,mención,[],general
3,3,Bernardo Diaz Martínez,2021-04-28 13:35:08+00:00,1387399988846776322,@publico_es No están ellos sacando todos los d...,"[12, 192]",['Twitter Web App'],0,0,0,87,310,Partido Popular,mención,[],general
4,4,Ritxi ·🔻·,2021-04-28 13:35:03+00:00,1387399968097509377,"@AngelBlancoMo @populares Bueno, yo hablaba de...","[26, 304]",['Twitter for Android'],0,0,63,346,1130,Partido Popular,mención,[],general
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48536,5,Enrique rm529x7,2021-04-27 23:58:25+00:00,1387194454021492740,@Luis44544014 @Teofilo1952 Y en Madrid donde l...,"[27, 108]",Twitter Web App,0,1,97,140,4042,NaN,NaN,NaN,NaN
48537,6,Antonio Ruiz Vaz,2021-04-27 23:58:24+00:00,1387194448547913730,Después de oír Ayuso y Monasterio todas las ba...,"[0, 280]",Twitter for Android,0,0,112,180,2895,NaN,NaN,NaN,NaN
48538,7,LucasRCDE 🇪🇸🎲,2021-04-27 23:58:18+00:00,1387194424795467778,"Tengo la teoría de que, evidentemente, lo de A...","[0, 205]",Twitter for Android,0,0,436,196,16035,NaN,NaN,NaN,NaN
48539,8,Santi Orozco 🧡🇪🇸 🇪🇺,2021-04-27 23:58:11+00:00,1387194396702121988,@mariviti1 Cs hizo la moción de censura en Mur...,"[11, 288]",Twitter for Android,1,1,2039,2702,31750,NaN,NaN,NaN,NaN


In [113]:
pd.read_csv('../../../../sarahr/Ironhack/political_spanish_sentiment/political_spanish_sentiment_scripts/political_spanish_sentiment_mentions.csv')

,Unnamed: 0,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,partido,tipo de post,hashtags,campaña
0,0,jeroma 🇪🇸,2021-04-27 23:59:51+00:00,1387194817361416197,@DebatAlRojoVivo Javier Maroto eres el más mi...,"[17, 102]",Twitter for iPhone,0,0,82,237,7134,Partido Popular,mención,[],general
1,1,☀️ PaulaMIX 💜,2021-04-27 23:59:10+00:00,1387194644564529164,"@DBofetadas dilo sin llorar, al final sois mas...","[12, 94]",Twitter Web App,0,0,1996,4987,16123,Partido Popular,mención,[],general
2,2,🇪🇸🏳️‍🌈 Miguel Ángel Bastenier,2021-04-27 23:57:46+00:00,1387194289134919688,"☝️💙🖋🖋 “EL PEOR ENEMIGO DEL SOCIO-COMUNISMO, NO...","[0, 244]",Twitter for iPhone,0,1,447,1877,45837,Partido Popular,mención,"['MargaretTHATCHER', 'YoConAyuso', 'Libertad']",general
3,3,Un transeúnte,2021-04-27 23:57:33+00:00,1387194238350368768,@mangemalo @populares @IdiazAyuso Sabes que es...,"[34, 65]",Twitter for Android,0,0,57,133,2011,Partido Popular,mención,[],general
4,4,Un transeúnte,2021-04-27 23:56:22+00:00,1387193936784076805,@ArwenCelta @populares @IdiazAyuso Te refieres...,"[35, 65]",Twitter for Android,0,0,57,133,2011,Partido Popular,mención,[],general
5,5,unocualquiera 🇪🇸,2021-04-27 23:56:11+00:00,1387193891351404544,@guille2702 @juanjodom @PODEMOS @PSOE @vox_es ...,"[70, 94]",Twitter Web App,0,3,106,173,7271,Partido Popular,mención,[],general
6,6,🇪🇸💚Prohibido👑Rendirse💙🇪🇸,2021-04-27 23:55:25+00:00,1387193698128105477,@AlmeidaPP_ Que grande eres de lo poco que val...,"[12, 69]",Twitter for Android,0,0,10134,11133,18550,Partido Popular,mención,[],general
7,7,Santiago Mariscal,2021-04-27 23:55:16+00:00,1387193663680393217,Ciudadanos y Vox se van del pleno al negarse P...,"[0, 140]",Twitter Web App,0,1,3056,3275,44924,Partido Popular,mención,[],general
8,8,Aquelo,2021-04-27 23:55:13+00:00,1387193647842615298,@adelillay @MariconLiberta1 Y quien le dice qu...,"[28, 280]",Twitter for Android,0,0,61,96,16655,Partido Popular,mención,[],general
9,9,JMD,2021-04-27 23:53:42+00:00,1387193266542718976,@aus29851124 @CorreosAtiende @iBarbarellah CC ...,"[43, 266]",Twitter Web App,3,6,188,720,256,Partido Popular,mención,[],general
